In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime
import os

# Setup the Chrome driver
driver = webdriver.Chrome()

try:
     
    driver.get('https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'cboCategorias')))

    
    dropdown = driver.find_element(By.XPATH, '/html/body/form[1]/div[3]/div/fieldset/div[5]/div[1]/div[1]/div/a')
    dropdown.click()

    categoria_dropdown = Select(driver.find_element(By.ID, 'cboCategorias'))
    actions = ActionChains(driver)
    actions.move_to_element(dropdown).click().perform()
    
    time.sleep(3)
    
    search = driver.find_element("xpath",'/html/body/form[1]/div[3]/div/fieldset/div[5]/div[1]/div[1]/div/div/div/input')
    search.send_keys("Fato Relevante")
    search.send_keys(Keys.ENTER)    
    
    #Seleciona a data para semana
    #data_entrega_dropdown = Select(driver.find_element(By.ID, 'cboEntrega'))
    #data_entrega_dropdown.select_by_visible_text('Na semana')
    #semanal = driver.find_element(By.XPATH, '/html/body/form[1]/div[3]/div/fieldset/div[4]/div[1]/div[1]/div[2]/input')
    #semanal.click()
    
    submit_button = driver.find_element(By.ID, 'btnConsulta')
    submit_button.click()
    time.sleep(10)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'grdDocumentos')))

    # Print out the number of rows found
    print(f"Number of rows found: {len(rows)}")

    # Define the time range
    start_time = datetime.strptime("10:00", "%H:%M").time()
    end_time = datetime.strptime("17:00", "%H:%M").time()

    for row in rows:
        columns = row.find_elements(By.XPATH, './td')
        
        # Print the text content of the row
        print(f"Row text: {row.text}")
        
        # Skip rows with insufficient columns
        if len(columns) < 5:
            print(f"Skipping row with insufficient columns: {len(columns)} columns found.")
            continue

        # Extract datetime string (assumed to be in the 3rd column)
        datetime_str = columns[2].text
        
        # Check for valid datetime string
        if ' ' not in datetime_str:
            print(f"Skipping row with invalid datetime format: {datetime_str}")
            continue

        try:
            # Split the datetime string to separate the date and time
            date_part, time_part = datetime_str.split()
            time_obj = datetime.strptime(time_part, "%H:%M").time()
        except ValueError as e:
            print(f"Skipping row due to ValueError: {e}")
            continue

        # Check if the time is within the specified range
        if start_time <= time_obj <= end_time:
            # Extract "Categoria" (assumed to be in the 1st column)
            categoria = columns[0].text

            # Extract "Data Referência" (assumed to be in the 2nd column)
            data_referencia = columns[1].text

            # Check if the download link exists in the 5th column
            try:
                download_link = columns[4].find_element(By.XPATH, './a').get_attribute('href')
            except Exception as e:
                print(f"Skipping row due to missing download link: {e}")
                continue

            # Print the information
            print(f"Categoria: {categoria}, Data Referência: {data_referencia}, Time: {time_part}")

            # Download the document
            driver.get(download_link)
            time.sleep(5)  # Wait for the download to complete (adjust as necessary)

finally:
    driver.quit()


The chromedriver version (124.0.6367.207) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (125.0.6422.77); currently, chromedriver 125.0.6422.78 is recommended for chrome 125.*, so it is advised to delete the driver in PATH and retry


Number of rows found: 187


MaxRetryError: HTTPConnectionPool(host='localhost', port=58285): Max retries exceeded with url: /session/0c5a53800301e2b7469b79c9d933b624/element/f.E048ADBDE4B3D0625692424930D0E377.d.3885345D5E32643B2CA84A1D764AAA1C.e.122/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10b8b7a90>: Failed to establish a new connection: [Errno 61] Connection refused'))